In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
from pathlib import Path

In [4]:
# TODO: PATH 설정

# main project path
PROJECT_ROOT = Path('/content/drive/Othercomputers/my_notebook/lion_final_pro_multimodal-anomaly-report-generation')
os.chdir(PROJECT_ROOT) # 현재 경로 수정
# print(f"현재 작업 경로: {os.getcwd()}")

# dataset path
DATA_ROOT = PROJECT_ROOT / "dataset" / "MMAD"
DOMAIN_JSON = DATA_ROOT / "domain_knowledge.json"
MMAD_JSON = DATA_ROOT / "mmad.json"
META_CSV = DATA_ROOT / "metadata.csv"

# config.yaml path
CONFIG_ROOT = PROJECT_ROOT / "configs"
RUNTIME_CONFIG_ROOT = CONFIG_ROOT / "runtime.yaml"
EVAL_CONFIG_ROOT = CONFIG_ROOT / "eval.yaml"

# check (선택사항)
print(f"Project Root: {PROJECT_ROOT}")
print(f"Data Root: {DATA_ROOT}")
print(f"Config Root: {CONFIG_ROOT}")
print(f"Current working directory: {os.getcwd()}")

Project Root: /content/drive/Othercomputers/my_notebook/lion_final_pro_multimodal-anomaly-report-generation
Data Root: /content/drive/Othercomputers/my_notebook/lion_final_pro_multimodal-anomaly-report-generation/dataset/MMAD
Config Root: /content/drive/Othercomputers/my_notebook/lion_final_pro_multimodal-anomaly-report-generation/configs
Current working directory: /content/drive/Othercomputers/my_notebook/lion_final_pro_multimodal-anomaly-report-generation


In [7]:
!python ./scripts/setup_conda.py

🌟 Anomalib + AI Models 통합 설치 (ToS Fix 적용)

2️⃣ 약관 동의 및 환경 생성 중...

🏃 실행 중: /content/conda/bin/conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/main
accepted Terms of Service for https://repo.anaconda.com/pkgs/main

🏃 실행 중: /content/conda/bin/conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/r
accepted Terms of Service for https://repo.anaconda.com/pkgs/r

🏃 실행 중: /content/conda/bin/conda create -n anomaly_report python=3.10 -y -q
Jupyter detected...
2 channel Terms of Service accepted
Retrieving notices: ...working... done
Channels:
 - defaults
Platform: linux-64
Solving environment: ...working... done

## Package Plan ##

  environment location: /content/conda/envs/anomaly_report

  added / updated specs:
    - python=3.10


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    libnsl-2.0.0               |       h5eee18b_0          31 K

In [14]:
%%bash
/content/conda/envs/anomaly_report/bin/python <<EOF

import os
from pathlib import Path
from unittest.mock import MagicMock

# --- [초강수 패치: 시각화 에러 방지] ---
import lightning.pytorch.callbacks as callbacks
class FakeProgressBar(callbacks.Callback): pass
callbacks.RichProgressBar = FakeProgressBar

import anomalib.utils.rich as anomalib_rich
anomalib_rich.CacheRichLiveState = MagicMock()
anomalib_rich.safe_track = lambda sequence, *args, **kwargs: sequence
# --------------------------------------

from anomalib.data import MVTec
from anomalib.models import EfficientAd
from anomalib.engine import Engine
from lightning.pytorch.loggers import CSVLogger

# --- [사용자 설정 구간] ---
DATA_ROOT = Path('/content/drive/Othercomputers/my_notebook/lion_final_pro_multimodal-anomaly-report-generation/dataset/MMAD/MVTec-AD')
CATEGORY = "bottle"
RESULT_DIR = Path('/content/drive/Othercomputers/my_notebook/lion_final_pro_multimodal-anomaly-report-generation/notebooks/noh/results')

def main():
    # 1. 데이터 모듈
    datamodule = MVTec(
        root=DATA_ROOT,
        category=CATEGORY,
        train_batch_size=1,
        eval_batch_size=32,
        image_size=(256, 256),
        num_workers=2
    )

    # 2. 모델 설정
    model = EfficientAd(model_size="small")

    if isinstance(model.model_size, str):
        class StrWrapper(str):
            @property
            def value(self): return self
        model.model_size = StrWrapper(model.model_size)

    # 3. 엔진 및 로거 설정
    csv_logger = CSVLogger(save_dir=RESULT_DIR, name=f"efficientad_{CATEGORY}")

    engine = Engine(
        task="segmentation",
        default_root_dir=RESULT_DIR / "efficientad" / CATEGORY,
        enable_progress_bar=False,
        logger=csv_logger,
        max_epochs=10
    )

    print(f"🚀 [{CATEGORY}] EfficientAd 분석 시작... (Dynamic Wrapper 적용)")

    try:
        engine.fit(model=model, datamodule=datamodule)
        results = engine.test(model=model, datamodule=datamodule)

        print("\n" + "="*50)
        print(f"✅ EfficientAd {CATEGORY} 결과:")
        print(results)
        print("="*50)

    except Exception as e:
        print(f"❌ 에러 발생: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()
EOF

🚀 [bottle] EfficientAd 분석 시작... (Dynamic Wrapper 적용)
┏━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name                  ┃ Type                     ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ model                 │ EfficientAdModel         │  8.1 M │
│ 1 │ _transform            │ Compose                  │      0 │
│ 2 │ normalization_metrics │ MinMax                   │      0 │
│ 3 │ image_threshold       │ F1AdaptiveThreshold      │      0 │
│ 4 │ pixel_threshold       │ F1AdaptiveThreshold      │      0 │
│ 5 │ image_metrics         │ AnomalibMetricCollection │      0 │
│ 6 │ pixel_metrics         │ AnomalibMetricCollection │      0 │
└───┴───────────────────────┴──────────────────────────┴────────┘
Trainable params: 8.1 M                                                         
Non-trainable params: 0                                                         
Total params: 8.1 M                                        

/content/conda/envs/anomaly_report/lib/python3.10/site-packages/lightning/fabric/__init__.py:40: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
OpenVINO is not installed. Please install OpenVINO to use OpenVINOInferencer.
Could not find wandb. To use this feature, ensure that you have wandb installed.
Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: /content/drive/Othercomputers/my_notebook/lion_final_pro_multimodal-anomaly-report-generation/notebooks/noh/results/efficientad_bottle
F1Score class exist